In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

# Tensor Values
The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's rank is its number of dimensions, while its shape is a tuple of integers specifying the array's length along each dimension

In [0]:
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

# TensorFlow Core Walkthrough
You might think of TensorFlow Core programs as consisting of two discrete sections:  

1 Building the computational graph (a tf.Graph).  
2 Running the computational graph (using a tf.Session   

 #  1)  Graph
A computational graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.  

tf.Operation (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.   
tf.Tensor: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.  

In [0]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


Notice that printing the tensors does not output the values 3.0, 4.0, and 7.0 as you might expect. The above statements only build the computation graph



# 2 ) TensorBoard
TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

First you save the computation graph to a TensorBoard summary file as follows:

In [0]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()



'''
This will produce an event file in the current directory with a name in the following format:

events.out.tfevents.{timestamp}.{hostname}

Now, in a new terminal, launch TensorBoard with the following shell command:

tensorboard --logdir .
'''

# 3) Session
To evaluate tensors, instantiate a tf.Session object, informally known as a session. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a tf.Graph is like a .py file, a tf.Session is like the python executable.

The following code creates a tf.Session object and then invokes its run method to evaluate the total tensor we created above:

In [0]:
sess = tf.Session()
print(sess.run(total))

7.0


You can pass multiple tensors to tf.Session.run. The run method transparently handles any combination of 
#tuples or  
#dictionaries

In [0]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


During a call to tf.Session.run any tf.Tensor only has a single value. For example, the following code calls tf.random_uniform to produce a tf.Tensor that generates a random 3-element vector (with values in [0,1)):

In [0]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.05108666 0.9311749  0.6947621 ]
[0.19103205 0.08454764 0.65318286]
(array([1.3837353, 1.9445282, 1.065348 ], dtype=float32), array([2.3837352, 2.944528 , 2.0653481], dtype=float32))


# 4)Feeding or placeholder
As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later, like a function argument.

In [0]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

The preceding three lines are a bit like a function in which we define two input parameters (x and y) and then an operation on them. We can evaluate this graph with multiple inputs by using the
#feed_dict   
argument of the tf.Session.run method to feed concrete values to the placeholders:

In [0]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


# Datasets
Placeholders work for simple experiments, but tf.data are the preferred method of streaming data into a model.

To get a runnable tf.Tensor from a Dataset you must first convert it to a tf.data.Iterator, and then call the Iterator's tf.data.Iterator.get_next method.

The simplest way to create an Iterator is with the tf.data.Dataset.make_one_shot_iterator method. For example, in the following code the next_item tensor will return a row from the my_data array on each run call:

In [0]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

Reaching the end of the data stream causes Dataset to throw an tf.errors.OutOfRangeError. For example, the following code reads the next_item until there is no more data to read:



In [0]:
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


In [0]:
# If the Dataset depends on stateful operations you may need to initialize the iterator before using it, as shown below:

r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

Instructions for updating:
Colocations handled automatically by placer.
[ 1.5601076  1.2840494 -0.7154319]
[ 1.4055388  -0.88288075 -1.4299309 ]
[ 1.0941852   1.277455   -0.10627846]
[ 1.129074   -0.58704346  0.32764482]
[-1.95634   -1.0096155  1.2705725]
[ 0.06963531 -1.1843501  -0.17169076]
[-0.8860986   0.92886597  1.0618613 ]
[0.09516091 0.19548108 1.0239028 ]
[ 0.20095097 -0.02072884  0.69004965]
[-0.15319447 -2.0367012  -2.371966  ]


# Layers
A trainable model must modify the values in the graph to get new outputs with the same input. tf.layers are the preferred way to add trainable parameters to a graph.

Layers package together both the variables and the operations that act on them. For example a densely-connected layer performs a weighted sum across all inputs for each output and applies an optional activation function. The connection weights and biases are managed by the layer object.

1))))))Creating Layers     
The following code creates a "tf.layers.Dense "layer that takes a batch of input vectors, and produces a single output value for each. To apply a layer to an input, call the layer as if it were a function. For example:

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

2)))))Initializing Layers  
The layer contains variables that must be initialized before they can be used. While it is possible to initialize variables individually, you can easily initialize all the variables in a TensorFlow graph as follows:



In [0]:
init = tf.global_variables_initializer()
sess.run(init)

3))))))Executing Layers     
Now that the layer is initialized, we can evaluate the linear_model's output tensor as we would any other tensor. For example, the following code:



In [0]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[-2.4294057]
 [-3.7012608]]


4))))))))))Layer Function shortcuts   
For each layer class (like tf.layers.Dense) TensorFlow also supplies a shortcut function (like tf.layers.dense). The only difference is that the shortcut function versions create and run the layer in a single call. For example, the following code is equivalent to the earlier version:



In [0]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[ 4.2711945]
 [10.986111 ]]


# Feature columns
The easiest way to experiment with feature columns is using the tf.feature_column.input_layer function. This function only accepts dense columns as inputs, so to view the result of a categorical column you must wrap it in an tf.feature_column.indicator_column.

In [0]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Running the inputs tensor will parse the features into a batch of vectors.

Feature columns can have internal state, like layers, so they often need to be initialized. Categorical columns use tf.contrib.lookup internally and these require a separate initialization op, tf.tables_initializer.

In [0]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [0]:
# Once the internal state has been initialized you can run inputs like any other tf.Tensor:

print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


# Training
Now that we are familiar with the basics of core TensorFlow, let's train a small regression model manually.

In [0]:
# Define the data
# First let's define some inputs, x, and the expected output for each input, y_true:

In [0]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

In [0]:
# Define the model
# Next, build a simple linear model, with 1 output:

In [0]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

 evaluate the predictions as follows:

In [0]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-1.022412]
 [-2.044824]
 [-3.067236]
 [-4.089648]]


In [0]:
# Loss
#To optimize a model, you first need to define the loss. We'll use the mean square error, a standard loss for regression problems.

#While you could do this manually with lower level math operations, the tf.losses module provides a set of common loss functions.
#You can use it to calculate the mean square error as follows:

In [0]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

1.115827


Training   
TensorFlow provides optimizers implementing standard optimization algorithms. These are implemented as sub-classes of tf.train.Optimizer. They incrementally change each variable in order to minimize the loss. The simplest optimization algorithm is gradient descent, implemented by tf.train.GradientDescentOptimizer. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [0]:
#This code builds all the graph components necessary for the optimization, and returns a training operation. When run, the training op will update variables in the graph. You might run it as follows:

for i in range(10):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

0.07254096
0.07210727
0.071676135
0.071247585
0.07082162
0.07039819
0.06997729
0.069558896
0.06914302
0.06872963


In [0]:
#Complete program

x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(10):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

print(sess.run(y_pred))

1.435248
1.0340528
0.75544333
0.5618957
0.42737144
0.33380374
0.2686564
0.2232306
0.19149043
0.16924791
[[-0.63954204]
 [-1.4118063 ]
 [-2.1840706 ]
 [-2.9563348 ]]
